In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Requesting details of inline valuation operations using the insights API

Attributes
----------
Insights
Request Logs
"""

toggle_code("Toggle Docstring")

# Requesting log details using the Insights API

This notebook demonstrates the basic usage of the Insights API to request logs and the detailed request/response bodies for a given operation. In this notebook, we will perform the following workflow:

- We run a valuation of an inline GBP portfolio with a reporting currency of JPY whereby we provide the requisite JPY/GBP FX rate and weighted instruments to ensure our valuation runs successfully.
- In the same portfolio, we attempt to run the valuation with a reporting currency of AUD but where we don't provide any  weighted instruments for the valuation. This valuation will fail.
- We then use the Insights Requests API to retreive all weighted instrument valuation requests from the previous 5 minutes for this session and display them along with any errors in a data frame.

The reason that an inline valuation is used in this instance is to make the notebook easier to re-run and that meaningful data can be extracted from the response body to indicate the success or failure of the operation.

## 1. Setup LUSID and Insights

This section will load the relevant Python packages as well as create the LUSID and Insights API clients. The LUSID API clients will be used to create the valuation whereas the Insights API client will be used to request the logs and request/response bodies. 

A unique correlation ID will also be set here when the Lusid API factory is constructed. The correlation ID is attached to all request logs created by the API object and will be used by the insights API later in the notebook to find logs related to this session. More can be found on correlation IDs under this [knowledge base article](https://support.lusid.com/knowledgebase/article/KA-01714/en-us)

In [ ]:
# Import Insights specific packages
from finbourne_insights import api as ia
import finbourne_insights
import fbnsdkutilities

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken

#Import system packages
import json
import pytz
from IPython.core.display import HTML
import uuid
from datetime import datetime, timedelta
import time
from flatten_json import flatten
import os
import pandas as pd
import math
from pprint import pprint
import backoff

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Get secrets path for API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Set a correlation ID in order to find all logs related to this session
correlation_id = str(uuid.uuid4())

# Initiate a Lusid API Factory which is the client side object for interacting with Lusid APIs
l_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
    correlation_id = correlation_id
)

# Load LUSID API Components
aggregation_api = l_api_factory.build(lusid.api.AggregationApi)
quotes_api = l_api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = l_api_factory.build(lusid.api.ConfigurationRecipeApi)

lusid_api_url = l_api_factory.api_client.configuration.host
insights_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "insights"

# Initiate an Insights API Factory which is the client side object for interacting with Insight APIs
i_api_factory = fbnsdkutilities.ApiClientFactory(
    finbourne_insights,
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    api_url = insights_api_url,
    app_name = "LusidJupyterNotebook"
)

# Build insight API components
i_requests_api = i_api_factory.build(ia.RequestsApi)

# Set Global scope
global_scope = "Insights_Inline_valuation_NB"

# Defining variables
valuation_date = datetime(year=2023, month=1, day=26, tzinfo=pytz.UTC).isoformat()

## 2. Load quotes

### 2.1 Instrument Value Quotes

Load 5 prices into the quotes store with associated instrument 'ClientInternal' identifiers. These will be linked to the weighted instruments created below in Section 3.

In [ ]:
values = [20, 50, 100, 75, 60]

for i in range(1,6):
    quotes_api.upsert_quotes(
        scope = global_scope,
        request_body = {f"quote_{i}": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=f"client_internal_{i}",
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid"             
                ),
                effective_at = valuation_date
            ),
            metric_value=models.MetricValue(
                value=values[i-1],
                unit="GBP"
            )
        )}        
    )

## 2.2 FX Rate quotes

Load FX rates required for FX rate inference into the quotes store.

In [ ]:
def spot_request(from_ccy, to_ccy, rate, valuation_date):
    return models.UpsertQuoteRequest(
               quote_id=models.QuoteId(
                   models.QuoteSeriesId(
                       provider='Lusid',
                       instrument_id=f'{from_ccy}/{to_ccy}',
                       instrument_id_type='CurrencyPair',
                       quote_type='Rate',
                       field='mid'
                   ),
                   effective_at=valuation_date
               ),
               metric_value=models.MetricValue(
                   value=rate,
                   unit=f'{from_ccy}/{to_ccy}'
               ),
               lineage='None'
    )

# JPY/GBP FX rate quote
response = quotes_api.upsert_quotes(scope=global_scope,
                                   request_body={"1": spot_request("JPY", "GBP", 0.006618, valuation_date)})                                  
# AUD/GBP FX rate quote
response = quotes_api.upsert_quotes(scope=global_scope,
                                   request_body={"1": spot_request("AUD", "GBP", 0.7319, valuation_date)})

## 3. Define Weighted Instruments

Define 5 weighted equity instruments with the same 'ClientInternal' identifiers as those of the 5 quotes, a quantity of 1 and a currency of GBP (same as for the quotes). Put these 5 definitions into a list to be passed into the successful inline valuation request in Section 5.

In [ ]:
weighted_instruments = []

for i in range(1, 6):
    weighted_instrument = lusid.WeightedInstrument(
        quantity=1,
        holding_identifier=f"client_internal_{i}",
        instrument=models.Equity(
            identifiers= lusid.EquityAllOfIdentifiers(
                client_internal=f"client_internal_{i}",
            ),
            dom_ccy="GBP",
            instrument_type="Equity",
        ), 
    )      
    
    weighted_instruments.append(weighted_instrument)

## 4. Create Valuation Recipe

Define a valuation recipe to use the quotes loaded into the quotes store. 

In [ ]:
# Create recipes
recipe_scope="Insights_Inline_valuation_NB"
recipe_code="Insights_Inline_valuation_NB"

# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            # Define how to resolve the quotes
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=global_scope,
                quote_type="Price",
                field="mid",
            ),
            models.MarketDataKeyRule(
                key='Fx.CurrencyPair.*',
                data_scope=global_scope,
                supplier='Lusid',
                quote_type='Rate',
                quote_interval='1D.0D',
                field="mid"
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope='Lusid',
            attempt_to_infer_missing_fx=True,
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

## 5. Run Inline Valuation Requests

Attempt to create valuation requests for the predefined weighted instruments with JPY & AUD report currencies.

### 5.1 Define Inline Valuation Request Function

In [ ]:
def generate_valuation_request(valuation_effectiveAt, report_currency, instruments):

    # Create the valuation request
    valuation_request = models.InlineValuationRequest(
        recipe_id=models.ResourceId(
            scope=recipe_scope, code=recipe_code
        ),
        metrics=[
            models.AggregateSpec("Valuation/PvInReportCcy", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy/Ccy", "Value"),
            models.AggregateSpec("Analytic/default/InstrumentTag", "Value"),
            models.AggregateSpec("Quotes/FxRate/DomReport", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
            models.AggregateSpec("Quotes/Price/Ccy", "Value")
        ],
        report_currency = report_currency,
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effectiveAt
        ),
        instruments=instruments
    )

    return valuation_request

### 5.2 Run Inline Valuation in JPY

This valuation will return successful results.

In [ ]:
# Run inline valuation in JPY
aggregation = aggregation_api.get_valuation_of_weighted_instruments(
    inline_valuation_request=generate_valuation_request(
        valuation_date, "JPY", weighted_instruments
    )
)

output = pd.DataFrame(aggregation.data)
output

### 5.3 Run Inline Valuation in AUD

Here we do not specify the weighted instruments and as such an API exception is returned.

In [ ]:
# Run inline Valuation for AUD with an empty list of instruments
try :
    aggregation = aggregation_api.get_valuation_of_weighted_instruments(
        inline_valuation_request=generate_valuation_request(
            valuation_date, "AUD", []
        )
    )
except lusid.ApiException as e:
    print("ERROR : " + json.loads(e.body)["title"])

## 6. Get the valuation requests using Insights

### 6.1 Get the request logs

Now that we have a successful and unsuccesful request in the system, we can request the logs which will give us an overview of each operation. Here we use the insights API's list_request_logs function which can request the top-level information from the request logs. We then filter the request for all weighted instrument valuations made in the last 5 minutes. We also provide the correlation ID which was set in section 1 to filter for only transactions from this session.

In [ ]:
# Logs are written asynchronously and there may be a lag for them to be queryable after writing. 
# This function will keep polling until it finds both logs for the 2 valuations from the previous steps.

@backoff.on_predicate(backoff.expo, lambda x: len(x.values) < 2, max_tries=5)
def insights_log_requests(operation, timestamp_diff_lower, timestamp_diff_upper):
    # Define filter
    timestamp_lower = (datetime.now() - timedelta(minutes=timestamp_diff_lower)).strftime(f"%Y-%m-%dT%H:%M:%S")
    timestamp_upper = (datetime.now() - timedelta(minutes=timestamp_diff_upper)).strftime(f"%Y-%m-%dT%H:%M:%S")

    filter= f"timestamp gt {timestamp_lower} and timestamp lt {timestamp_upper} and operation eq '{operation}' and CorrelationId eq '{correlation_id}' "

    # Request the logs that satisfy the filter
    return i_requests_api.list_request_logs(filter=filter)

i_response = insights_log_requests("GetValuationOfWeightedInstruments", 5, 0)

### 6.2 Get the details for each operation 

Using the logs which were retrieved in the previous section, we can take the Request IDs and construct a dataframe which displays the request/response bodies along with error details if the request was unsuccessful.

In [ ]:
# The request and response bodies are written asynchronously and seperate to the request logs, so they may be a further
# delay after the initial logs are retrieved.
# This function will keep trying to request the request and response bodies until neither request returns an exception.
@backoff.on_exception(backoff.expo, finbourne_insights.ApiException, max_tries=5 )
def request_response_reqs(id):
    request = i_requests_api.get_request(id)
    response = i_requests_api.get_response(id)
    
    return (request,response)

# Function to construct a data frame out of the request and response bodies.
def request_response_bodies(req_logs):

    rr_bodies = []

    for row in req_logs.values:
        req_resp = request_response_reqs(row.id)
        request_body = json.loads(req_resp[0].body)
        response_body = json.loads(req_resp[1].body)
        error = None
        error_details = None
        if row.http_status_code == 400 : 
            error  = response_body["name"]
            error_details = response_body["errorDetails"]
        rr_bodies.append({"id" : row.id,
                        "timestamp" : row.timestamp, 
                        "outcome" : row.outcome,
                        "error" : error,
                        "error_details" : error_details,
                        "request_body" : request_body,
                        "response_body" : response_body,})
    return rr_bodies

request_bodies_df = pd.DataFrame(request_response_bodies(i_response))
request_bodies_df

If we want to examine the errors further, we can drill down into the error details by walking through the JSON. This will vary based on the type of error returned, but here is an example for the InvalidParameterValue error returned above:

In [ ]:
for index, row in request_bodies_df.iterrows() :
    if row['error'] : 
        print(f"\
For message ID {row.id}, error type {row['error']} \
was returned for the {row['response_body']['errorDetails'][0]['id']} parameter. \
The error message was '{row['response_body']['errorDetails'][0]['detail']}'")